In [5]:
import pandas as pd
import numpy as np
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint vector (shape (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64) # Constraint coefficients matrix (shape (m, n))
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints

        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")

        # List to store sequence of vertices visited
        self.visited_vertices = []

    def isFeasible(self):
        """Check if the solution z is feasible."""
        return np.all(np.matmul(self.A, self.z) <= self.b + 1e-8)  # Added tolerance

    def __activeAndUnActiveConst(self):
        """Classify active and inactive constraints based on the current z."""
        constraint_values = np.matmul(self.A, self.z)
        tolerance = 1e-8
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]

        self.A1 = self.A[active_indices]
        self.B1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.B2 = self.b[inactive_indices]

    def __isVertex(self):
        """Check if the current solution z is a vertex."""
        self.__activeAndUnActiveConst()
        return self.A1.shape[0] == self.n

    def find_initial_vertex(self):
        """Find an initial vertex for the feasible region."""
        if not self.isFeasible():
            raise ValueError("Initial point z is not feasible.")

        while not self.__isVertex():
            self.__activeAndUnActiveConst()
            A1 = self.A1
            A2 = self.A2
            b2 = self.B2

            if A1.size == 0:
                # No active constraints, move towards the nearest constraint
                slack = self.b - np.dot(self.A, self.z)
                min_slack_index = np.argmin(slack)
                direction = self.A[min_slack_index].astype(np.float64)
                direction /= np.linalg.norm(direction)
            else:
                nullspace = np.array(Matrix(A1).nullspace())
                if nullspace.size == 0:
                    break  # At a vertex

                direction = nullspace[0].astype(np.float64).flatten()
                direction /= np.linalg.norm(direction)

            temp = np.dot(A2, direction)
            temp = np.where(np.abs(temp) < 1e-8, 1e-8, temp)
            alpha_values = (b2 - np.dot(A2, self.z)) / temp
            positive_alphas = [alpha for alpha in alpha_values if alpha > 0]
            if not positive_alphas:
                raise ValueError("Cannot find a positive alpha to move towards a vertex.")

            alpha = min(positive_alphas)
            self.z = self.z + alpha * direction

    def find_optimal_vertex(self):
        """Find the optimal vertex for the objective function and return the cost."""
        self.find_initial_vertex()
        self.visited_vertices.append((self.z.copy(), np.dot(self.c, self.z).item()))

        while True:
            self.__activeAndUnActiveConst()
            A1 = self.A1

            if A1.shape[0] != self.n:
                raise ValueError("Active constraint matrix A1 must be square.")

            try:
                A1_inv = np.linalg.inv(A1)
            except np.linalg.LinAlgError:
                raise ValueError("Active constraint matrix A1 is singular.")

            beta = np.dot(A1_inv.T, self.c)
            if np.all(beta >= -1e-8):
                final_cost = np.dot(self.c, self.z).item()
                return self.visited_vertices

            A1_inv_T = A1_inv.T
            u = None
            for x in A1_inv_T:
                if np.dot(-x, self.c) > 1e-8:
                    u = x
                    break

            if u is None:
                final_cost = np.dot(self.c, self.z).item()
                return self.visited_vertices

            temp = np.dot(self.A2, u)
            temp = np.where(np.abs(temp) < 1e-8, 1e-8, temp)
            alpha_values = (np.dot(self.A2, self.z) - self.B2) / temp
            positive_alphas = [alpha for alpha in alpha_values if alpha >= 0]
            if not positive_alphas:
                raise ValueError("Problem is unbounded.")

            alpha = min(positive_alphas)
            self.z = self.z - alpha * u

            if not self.isFeasible():
                raise ValueError("Solution became infeasible during iteration.")

            self.visited_vertices.append((self.z.copy(), np.dot(self.c, self.z).item()))


In [6]:
# Example Usage
filePath = '2.Dataset.csv'  # Path to the CSV file
simplex = SimplexAlgo(filePath)
results = simplex.find_optimal_vertex()
for vertex, cost in results:
    print(f"Vertex: {vertex}, Objective Value: {cost}")

Vertex: [0. 1.], Objective Value: -4.0
Vertex: [1. 0.], Objective Value: -3.0
